In [22]:
# define a model using pytorch
# a pretrained model adding layers to classify the 4 classes of the images
import torch.nn as nn
from torchvision import models
from torchvision.models import squeezenet1_1
from torchvision.models.squeezenet import SqueezeNet1_1_Weights



class MIDmodel(nn.Module):
    def __init__(self):
        super(MIDmodel, self).__init__()
        self.pretrained = squeezenet1_1()
        self.fc1 = nn.Linear(1000, 128)
        self.fc2 = nn.Linear(128, 4)
        self.dropout = nn.Dropout(0.7)

    def forward(self, x):
        x = self.pretrained(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [23]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, root_dir, patients_ids, patients_df, transform=None):
        # select the rows of the dataframe that correspond to the patients in the list
        self.patients_df = patients_df[patients_df["patient_id"].isin(patients_ids)]
        self.root_dir = root_dir
        self.idxs = set()
        if transform:
            self.transform = transform
            # duplicate the patients_df, so the network will see each image twice, one standard and one transformed
            # self.patients_df = pd.concat([self.patients_df] * 2, ignore_index=True)
        else:
            self.transform = None
               
            
    def __len__(self):
        return len(self.patients_df)

    def __getitem__(self, idx):
        # get idx row of the dataframe
        row = self.patients_df.iloc[idx]
        # path = root_dir/patientid_examid_spot_framenumber_score.png
        path = os.path.join(self.root_dir, row["patient_id"] + "_" + row["exam_id"] + "_" + row["spot"] + "_" + row["frame_number"] + "_" + row["score"] + ".png")
        image = Image.open(path)
        label = row["score"]
        # if the image is grayscale, convert it to RGB (used to test with the smaller dataset)
        # if image.mode != "RGB":
        #     image = image.convert("RGB")
               
        if self.transform:
            image = self.transform(image)
            # if path not in self.idxs: # if it's the first time we see the image, add its path to the set
            #     self.idxs.add(path)
            # else: # if it's the second time we see the image, apply the transform and remove its path from the set 
            #     image = self.transform(image)
            #     self.idxs.remove(path)
        # in any case, transform to tensor
        image = transforms.ToTensor()(image)
        image = transforms.Resize((224, 224))(image)
        # Finally the values are first rescaled to [0.0, 1.0] and then normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
        image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
        return image, int(label)

In [24]:
import pandas as pd
import torch
import glob
data_dir = "images/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
images_paths = glob.glob(f"{data_dir}*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[len(data_dir):-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)
patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)

# select the 8 patients for training based on the most balanced distribution of the scores
import itertools
combs = list(itertools.combinations(patients_ids, 8))
stds = []
for i, c in enumerate(combs):
    stds.append((images_df[images_df["patient_id"].isin(c)].groupby("score").count()["patient_id"].std(), i))

train_patients = [x for x in combs[min(stds)[1]]]
print(train_patients)
print(images_df[images_df["patient_id"].isin(train_patients)].groupby("score").count()["patient_id"])
test_patients = [x for x in patients_ids if x not in train_patients]
print(test_patients)
print(images_df[images_df["patient_id"].isin(test_patients)].groupby("score").count()["patient_id"])


['1051', '1017', '1050', '1052', '1069', '1048', '1067', '1047']
score
0    7888
1    7540
2    7189
3    7592
Name: patient_id, dtype: int64
['1066', '1068', '1045']
score
0    6625
1    1721
2    5535
3    3631
Name: patient_id, dtype: int64


transformations from the paper Deep Learning for Classification and Localization of COVID-19 Markers in Point-of-Care Lung Ultrasound
were each activated on the image-label pair with a probability of 0.33. The set of augmentation functions, each applied with a randomly sampled strength bounded by a set maximum, consists of:
- affine transformations (translation (max. ±15%), rotation (max. ±15◦), scaling (max. ±45%), and shearing (max. ±4.5◦))
- multiplication with a constant (max. ±45%)
- Gaussian blurring (σmax = 3/4 )
- contrast distortion (max. ±45%)
- horizontal flipping (p = 0.5)
- additive white Gaussian noise (σmax = 0.015)

In [25]:
transformations = transforms.Compose([
    transforms.RandomApply([transforms.RandomAffine(degrees=15, translate=(0.15, 0.15), scale=(0.55, 1.45), shear=4.5)], p=1/3),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.45, contrast=0.45, saturation=0.45, hue=0.45)], p=1/3),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.75))], p=1/3),
    transforms.RandomApply([transforms.RandomHorizontalFlip(p=0.5)], p=1/3)
    ])

In [26]:
from torch import optim
from tqdm import tqdm

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects/ len(dataloader.dataset)
    return epoch_loss, epoch_acc

def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [27]:
data_dir = "images/"
num_epochs = 10
learning_rate = 0.00001
batch_size = 8

model = MIDmodel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train_dataset = ImageDataset(data_dir, train_patients, images_df, transformations)
test_dataset = ImageDataset(data_dir, test_patients, images_df)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
results_train = []
results_test = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
    results_train.append([train_loss, train_acc])
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(model, test_dataloader, criterion, device)
    results_test.append([test_loss, test_acc])
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f"MIDmodel_{epoch}.pt")
    if epoch % 2 == 1:
        learning_rate = learning_rate/2
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Train: ", results_train)
print("Test: ", results_test)

# save the model
torch.save(model.state_dict(), f"MIDmodel.pt")

Epoch 0


100%|██████████| 3777/3777 [17:01<00:00,  3.70it/s]


Training Loss: 1.4453 Training Accuracy: 0.2713


100%|██████████| 2189/2189 [04:26<00:00,  8.21it/s]


Test Loss: 1.3651 Test Accuracy: 0.3936
Epoch 1


...

Test Loss: 1.4515 Test Accuracy: 0.4854
Epoch 9


100%|██████████| 3777/3777 [16:44<00:00,  3.76it/s]


Training Loss: 0.5772 Training Accuracy: 0.7737


100%|██████████| 2189/2189 [03:58<00:00,  9.18it/s]

Test Loss: 1.5062 Test Accuracy: 0.4978
Train:  [[1.4453151551324965, tensor(0.2713, device='cuda:0')], [1.278736829047383, tensor(0.4056, device='cuda:0')], [1.0870531683799356, tensor(0.5218, device='cuda:0')], [0.923765770988631, tensor(0.6007, device='cuda:0')], [0.8183876516514437, tensor(0.6576, device='cuda:0')], [0.7402640066483773, tensor(0.6972, device='cuda:0')], [0.6809365507032136, tensor(0.7247, device='cuda:0')], [0.6463492192383963, tensor(0.7430, device='cuda:0')], [0.6037752303301651, tensor(0.7579, device='cuda:0')], [0.5771569872453134, tensor(0.7737, device='cuda:0')]]
Test:  [[1.3650751340982847, tensor(0.3936, device='cuda:0')], [1.3009044412419697, tensor(0.4223, device='cuda:0')], [1.507390807594214, tensor(0.3301, device='cuda:0')], [1.6292844691393364, tensor(0.4055, device='cuda:0')], [1.5698632373725132, tensor(0.4069, device='cuda:0')], [1.5488062066295345, tensor(0.4343, device='cuda:0')], [1.4955579021049161, tensor(0.4344, device='cuda:0')], [1.75228601

Classifier to predict if the confidence is high enough, created on the entire dataset

In [36]:
# load the last model
model = MIDmodel()
model.load_state_dict(torch.load("MIDmodel.pt"))
model.to(device)
model.eval()

correct_preds = pd.DataFrame(columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])
wrong_preds = pd.DataFrame(columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])

entire_dataset = ImageDataset(data_dir, patients_ids, images_df)

for i in tqdm(range(len(entire_dataset))):
    img, label = entire_dataset[i]
    img = img.unsqueeze(0)
    img = img.to(device)
    output = model(img)
    sm = nn.Softmax(dim=1)
    sm = sm(output)
    sm = sm.cpu().detach().numpy()
    sm = sm[0]
    if torch.argmax(output) == label:
        correct_preds = pd.concat([correct_preds, pd.DataFrame([[label, torch.argmax(output).cpu().detach().numpy(), sm[0], sm[1], sm[2], sm[3]]], columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])])
    else:
        wrong_preds = pd.concat([wrong_preds, pd.DataFrame([[label, torch.argmax(output).cpu().detach().numpy(), sm[0], sm[1], sm[2], sm[3]]], columns=["true", "predicted", "sm0", "sm1", "sm2", "sm3"])])

correct_preds.to_csv("correct_preds.csv", index=False)
wrong_preds.to_csv("wrong_preds.csv", index=False)


100%|██████████| 2760/2760 [00:48<00:00, 56.65it/s]


In [56]:

class MIDbinary(nn.Module):
    def __init__(self):
        super(MIDbinary, self).__init__()
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.softmax(x)
        return x
    
# BinaryDataset
class BinaryDataset(Dataset):
    def __init__(self, preds):
        # add a clumn to correct_preds and wrong_preds that says if it's correct or not
        self.preds = preds

    def __len__(self):
        return len(self.preds)

    def __getitem__(self, idx):
        label = self.preds.iloc[idx]["correct"]
        sms = self.preds.iloc[idx][["sm0", "sm1", "sm2", "sm3"]]
        sms = torch.tensor(list(sms.values), dtype=torch.float32)
        return sms, int(label)

# train the binary model
binary_model = MIDbinary()
binary_model = binary_model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0000001
optimizer = optim.Adam(binary_model.parameters(), lr=learning_rate)

correct_preds = pd.read_csv("correct_preds.csv")
wrong_preds = pd.read_csv("wrong_preds.csv")

# add a clumn to correct_preds and wrong_preds that says if it's correct or not
correct_preds["correct"] = 1
wrong_preds["correct"] = 0
# stack the two dataframes
preds_total = pd.concat([correct_preds, wrong_preds])
# remove true and predicted columns
preds_total = preds_total.drop(["true", "predicted"], axis=1)
# save the values
preds_total.to_csv("values.csv", index=False)

# divide train and test data
train_size_bin = int(0.8 * (len(preds_total)))
test_size_bin = len(preds_total) - train_size_bin
# TODO: better split
from torch.utils.data import random_split
train_dataset_bin, test_dataset_bin = random_split(BinaryDataset(preds_total), [train_size_bin, test_size_bin])

train_dataloader_bin = DataLoader(train_dataset_bin, batch_size=batch_size, shuffle=True)
test_dataloader_bin = DataLoader(test_dataset_bin, batch_size=batch_size, shuffle=False)

results_train = []
results_test = []
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train_model(binary_model, train_dataloader_bin, optimizer, criterion, device)
    results_train.append([train_loss, train_acc.item()])
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(binary_model, test_dataloader_bin, criterion, device)
    results_test.append([test_loss, test_acc.item()])
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    if epoch % 2 == 1:
        learning_rate = learning_rate/2
        optimizer = optim.Adam(binary_model.parameters(), lr=learning_rate)

print("Train: ", results_train)
print("Test: ", results_test)

import matplotlib.pyplot as plt
# subplots with train[0], train[1], test[0], test[1]
fig, axs = plt.subplots(2, 2)
axs[0, 0].plot([i[0] for i in results_train])
axs[0, 0].set_title("Train Loss")
axs[0, 1].plot([i[1] for i in results_train])
axs[0, 1].set_title("Train Accuracy")
axs[1, 0].plot([i[0] for i in results_test])
axs[1, 0].set_title("Test Loss")
axs[1, 1].plot([i[1] for i in results_test])
axs[1, 1].set_title("Test Accuracy")
plt.show()



Epoch 0


100%|██████████| 276/276 [00:01<00:00, 165.11it/s]


Training Loss: 0.7122 Training Accuracy: 0.1187


100%|██████████| 69/69 [00:00<00:00, 270.97it/s]


Test Loss: 0.7153 Test Accuracy: 0.0851
Epoch 1


...

Training Loss: 0.7118 Training Accuracy: 0.1259


100%|██████████| 69/69 [00:00<00:00, 236.55it/s]


Test Loss: 0.7149 Test Accuracy: 0.0851
Epoch 9


 52%|█████▏    | 144/276 [00:00<00:00, 169.03it/s]


KeyboardInterrupt: 

In [50]:
model = MIDmodel()
model.load_state_dict(torch.load("MIDmodel.pt"))
model.to(device)
model.eval()

# get the predictions for the test set and pass it to the binary model
zero_count = 0
one_count = 0
for i in tqdm(range(len(test_dataset))):
    img, label = test_dataset[i]
    img = img.to(device)
    img = img.unsqueeze(0)
    output = model(img)
    sm = torch.nn.functional.softmax(output, dim=1)
    print(sm)
    binary_output = binary_model(sm)
    predicted_label = torch.argmax(output).cpu().detach().numpy()
    predicted_binary = torch.argmax(binary_output).cpu().detach().numpy()
    if predicted_binary == 0:
        zero_count += 1
    else:
        one_count += 1
    print(f"True label: {label} Predicted label: {predicted_label} Binary output: {predicted_binary}")
    if i == 10:
        break

print("Zero count: ", zero_count)
print("One count: ", one_count)

    


  0%|          | 10/17512 [00:00<05:29, 53.06it/s]

tensor([[9.7090e-01, 2.8933e-02, 1.6503e-04, 4.9146e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
True label: 0 Predicted label: 0 Binary output: 1
tensor([[0.8456, 0.1463, 0.0071, 0.0010]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
True label: 0 Predicted label: 0 Binary output: 1
tensor([[0.8400, 0.1505, 0.0080, 0.0015]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
True label: 0 Predicted label: 0 Binary output: 1
tensor([[0.7110, 0.2551, 0.0271, 0.0068]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
True label: 0 Predicted label: 0 Binary output: 1
tensor([[0.4896, 0.3659, 0.0943, 0.0502]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
True label: 0 Predicted label: 0 Binary output: 1
tensor([[0.1935, 0.3295, 0.1501, 0.3269]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
True label: 0 Predicted label: 1 Binary output: 1
tensor([[0.2936, 0.4087, 0.1444, 0.1534]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
True label: 0 Predi

t-SNE to visualize only the correct prediction (but still in both train and test)

In [12]:
# impiort TNSE
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

plt.show()
